In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import PIL
import glob

In [2]:
print('Current working directory: ',os.getcwd())

Current working directory:  C:\Users\Shimul\Documents\Building CSE 4000 Thesis


## Dataset path location using glob

In [3]:
building_house=glob.glob('augmented_dataset/building_house/*.*')
mosque=glob.glob('augmented_dataset/mosque/*.*')
temple=glob.glob('augmented_dataset/temple/*.*')
tinshed=glob.glob('augmented_dataset/tinshed/*.*')


In [4]:
print('Building house data : ',len(building_house))
print('Mosque data: ',len(mosque))
print('Temple data: ',len(temple))
print('Tinshed data: ',len(tinshed))

Building house data :  951
Mosque data:  953
Temple data:  962
Tinshed data:  951


In [5]:
dataset_classes=[building_house,mosque,temple,tinshed]
total_class=len(dataset_classes)
print('Total dataset class: ',total_class)

Total dataset class:  4


## Make dataset and labels using keras.preprocessing.image

In [6]:
from tensorflow.keras.preprocessing import image

In [7]:
data=[]
labels=[]

In [8]:
print('Data length: ',len(data))
print('labels length: ',len(labels))

Data length:  0
labels length:  0


In [9]:
# Building house
for i in building_house:
    img=image.load_img(i)
    img=image.img_to_array(img)
    data.append(img)
    labels.append(0)

In [10]:
#mosque
for i in mosque:
    img=image.load_img(i)
    img=image.img_to_array(img)
    data.append(img)
    labels.append(1)

In [ ]:
#temple
for i in temple:
    img=image.load_img(i)
    img=image.img_to_array(img)
    data.append(img)
    labels.append(2)

In [22]:
#tinshed
for i in tinshed:
    img=image.load_img(i)
    img=image.img_to_array(img)
    data.append(img)
    labels.append(3)

## Length of the data and types

In [23]:
print('Total data: ',len(data))
print('Labels of data: ',len(labels))

Total data:  5687
Labels of data:  5687


In [24]:
print('Data type: ',type(data))
print('Labels type: ',type(labels))

Data type:  <class 'list'>
Labels type:  <class 'list'>


## Convert the list into numpy array

In [25]:
data_array=np.array(data)
labels_array=np.array(labels)

In [26]:
print('Data type: ',type(data_array))
print('Labels type: ',type(labels_array))

Data type:  <class 'numpy.ndarray'>
Labels type:  <class 'numpy.ndarray'>


## Data split using sklearn train_test_split

In [27]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test =train_test_split(data_array,labels_array,test_size=0.2,random_state=42)

In [ ]:
print('x train shape: ',x_train.shape)
print('x test shape: ',x_test.shape)
print('y train shape: ',y_train.shape)
print('y test shape: ',y_test.shape)

## plot random image and label
if range(0,255) ->  int then plt.imshow(img)

range(0.0,255.0) ->float scale the image into (0,1) by dividing 255 then plt.imshow()

In [ ]:
random_image=x_train[0]
random_image_labels=y_train[0]
print('image shape: ',random_image.shape)
print('Data types of image is : ',random_image.dtype)
print('image output label: ',random_image_labels)

In [ ]:
print('Label: ',random_image_labels)
plt.imshow(random_image/255)
plt.show()

## Normalize the data for training
x_test -> [0,1]


In [ ]:
print('Data range is [0,255]: ',x_train[0][0][0])

In [ ]:
x_test_normalize=x_test/255
x_train_normalize=x_train/255

In [ ]:
print('Data range is [0,1]: ',x_train_normalize[0][0][0])

In [ ]:
plt.figure(figsize=(20,20))
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(x_train_normalize[i])
    plt.title('category: '+str(y_train[i]))
plt.show()

## Categorial the labels using one hot encoding Keras

In [ ]:
from tensorflow.keras.utils import to_categorical


In [ ]:
print('Non categorical values : ',y_test[:10])

### categorical y_test and y_train

In [ ]:
y_test_categorical=to_categorical(y_test)
y_train_categorical=to_categorical(y_train)

In [ ]:
print('Categorical values : \n',y_test_categorical[:10])


## Define model architecture
### CNN requires x_test norlalize values [0,1] and y_test categorical values [0. 0. 1. 0.]

In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential

In [ ]:
model=Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(224,224,3),activation='relu',padding='same'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(total_class,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.layers

## Softmax activation fuction

three class labels will be integer encoded as 0, 1, and 2. Then encoded to vectors as follows:

- Class 0: [1, 0, 0]
- Class 1: [0, 1, 0]
- Class 2: [0, 0, 1]

The softmax output might look as follows, which puts the most weight on class 1 and less weight on the other classes.

[0.09003057 0.66524096 0.24472847] -> [0,1,0]

<b>equaltion</b> e^z/sum (e^z1,...,e^zn)

## Adam optimzer

Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.

According to Kingma et al., 2014, the method is "computationally efficient, has little memory requirement, invariant to diagonal rescaling of gradients, and is well suited for problems that are large in terms of data/parameters".

## Compile the model using loss and optimizer

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

## Train the model and save the model suing call backs fucntion

In [ ]:
import tensorflow

In [ ]:
tensorflow.config.run_functions_eagerly(True)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='building.model.best.hdf5', verbose=1 ,save_best_only=True)

In [ ]:
checkpointer.filepath

In [ ]:
x_test_normalize.shape

In [ ]:
y_test_categorical.shape

In [ ]:
history=model.fit(x_test_normalize,y_test_categorical,epochs=100,batch_size=32,steps_per_epoch=3,callbacks=[checkpointer])

## Testing data using test set

In [ ]:
score=model.evaluate(x_test_normalize,y_test_categorical)
score

In [ ]:
print('Accuracy of the model is : ',score[1]*100)

## plot accuracy and loss graph

In [ ]:
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.title('Training accuracy graph')
plt.plot(history.history['accuracy'])

In [ ]:
plt.xlabel('epochs')
plt.ylabel('loss')
plt.title('Loss training graph')
plt.plot(history.history['loss'])

## Evaluate the model using test image
- class 0: building house
- class 1: mosque
- class 2: temple
- class 4: tinshed

In [ ]:
random_image=image.load_img('tinshed19.png',target_size=(224,224))
plt.imshow(random_image)
random_image=image.img_to_array(random_image)
test_image=random_image.reshape((1,)+random_image.shape)
print(test_image.shape)
predicted_class=model.predict(test_image)
print(predicted_class)
print('True classification')

In [ ]:
random_image=image.load_img('building0.png',target_size=(224,224))
plt.imshow(random_image)
random_image=image.img_to_array(random_image)
test_image=random_image.reshape((1,)+random_image.shape)
print(test_image.shape)
predicted_class=model.predict(test_image)
print(predicted_class)
print('True classification')

In [ ]:
random_image=image.load_img('building1.png',target_size=(224,224))
plt.imshow(random_image)
random_image=image.img_to_array(random_image)
test_image=random_image.reshape((1,)+random_image.shape)
print(test_image.shape)
predicted_class=model.predict(test_image)
print(predicted_class)
print('True classification')

In [ ]:
random_image=image.load_img('mosque35.png',target_size=(224,224))
plt.imshow(random_image)
random_image=image.img_to_array(random_image)
test_image=random_image.reshape((1,)+random_image.shape)
print(test_image.shape)
predicted_class=model.predict(test_image)
print(predicted_class)
print('False classification')

## Classification report

In [ ]:
predicted_y=model.predict(x_test)
print(predicted_y.shape)

In [ ]:
y_test_categorical.shape

In [ ]:
predicted_y=np.argmax(predicted_y, axis=1)
predicted_y.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.metrics import classification_report
print('Classification report \n',classification_report(predicted_y,y_test))

## Save Model h5 format
This file includes
- Model architecture
- Model weight values during training
- model training config 
- optimizer 

In [ ]:
import time
t = time.time()

export_path_keras = "Saved Model/CNN_model_{}.h5".format(int(t))
print(export_path_keras)
model.save(export_path_keras)

## Reload the model h5 format

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model_path='Saved Model/CNN_model_1615634196.h5'
reload_model=load_model(model_path)
reload_model.summary()


## Experiment with reload model h5 format

In [ ]:
print(len(reload_model.weights))
print(reload_model.output_shape)

In [ ]:
reload_model.layers

In [ ]:
reload_model.weights